In [ ]:
from analysis.paper import *

In [ ]:
ags = data.load()
ags = data.with_times(ags)

In [ ]:
overleaf.table(parameters_table(ags), 'parameters')

In [ ]:
parameters_table(ags)

In [ ]:
raw = pd.Series({
        '$m_\text{scale}$': params['scale'][0],
        '$c_\text{scale}$': params['scale'][1],
        '$m_\text{center}$': params['center'][0],
        '$c_\text{center}$': params['center'][1],
        '$\text{height}$': params['height'].item()})
(raw
        .round(2)
        .to_frame().T
        .to_latex(index=False, label='parameters', caption='Fitted frontier parameters'))

In [ ]:
d = 600/np.log2(10)
1/(1 + 10**(-120/400))

In [ ]:
10**(120/600)